In [1]:
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import requests
import logging
import json
from collections import defaultdict
from datetime import datetime
import os

In [2]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

model = "llama-3.3-70b-versatile"

In [3]:
class check_prompt(BaseModel):
    """First LLM call: Extract basic accomodation information"""

    # description: str = Field(description="Raw description of the event")
    is_accomodation_search: bool = Field(
        description="Is the text a query related to finding an accomodation?"
    )


class EventDetails(BaseModel):
    """Second LLM call: Parse specific event details"""

    city: str = Field(description="City in which the accomodation should be booked")
    start_date: str = Field(
        description="Date and time of checking into the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )
    end_date: str = Field(
        description="Date and time of checking out of the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )

In [4]:
def check_accomodation_request(input_prompt: str):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"""
                    You are an assistant for finding accomodation. 
                    You are allowed to answer only to queries related to accomodation search.
                    In case of other requests, you must decline.
                    In this context, Analyze if the text describes a query for finding accomodation.
                    Say yes if the text describes a query for finding accomodation, else say no.
                    Output should be in json format:
                    {json.dumps(check_prompt.model_json_schema(), indent=2)}
                    """ 
            },
            {"role": "user", "content": input_prompt},
        ],
        response_format={"type": "json_object"}
    )

    return check_prompt.model_validate_json(completion.choices[0].message.content)


def extract_stay_details(input:str):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"""
                    You are an assistant for finding accomodation.
                    Extract the name of the city in which the user wishes to stay. If city is not found, strictly use the placeholder [None].
                    Extract the check-in date. Check-in date is date on which the user will start staying in the accomodation. Strictly use 'yyyy-mm-dd' date format. If check-in date is not found, strictly use the placeholder [None].
                    Extract the check-out date. Check-out date is date on which the user will leave the accomodation.  Strictly use 'yyyy-mm-dd' date format. If check-out date is not found, strictly use the placeholder [None].
                    Do not make any assumptions for the city, check-in date and check-out date.
                    Output should be in json format:
                    {json.dumps(EventDetails.model_json_schema(), indent=2)}
                """
            },
            {"role": "user", "content": input},
        ],
        response_format={"type": "json_object"}
    )

    return EventDetails.model_validate_json(completion.choices[0].message.content)

In [5]:
def process_accomodation_search(input: str):
    is_accomodation_request = check_accomodation_request(input)

    if not is_accomodation_request:
        return None
    else:
        stay_details = extract_stay_details(input)
        return stay_details

In [6]:
user_input = "Suggest me some hotels in Bengaluru for staying from 27th March 2025 and 31st March 2025"
# user_input = "Suggest me some hotels in Bengaluru"
# user_input = "Write a poem"

In [7]:
result = process_accomodation_search(user_input)

if not result:
    print("This is not an accomodation search request!")

In [8]:
result

EventDetails(city='Bengaluru', start_date='2025-03-27', end_date='2025-03-31')

In [9]:
place_of_stay = result.city
check_in_date = result.start_date
check_out_date = result.end_date

In [10]:
place_of_stay, check_in_date, check_out_date

('Bengaluru', '2025-03-27', '2025-03-31')

In [11]:
def get_weather_forecast(latitude, longitude, start_date, end_date):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,rain&start_date={start_date}&end_date={end_date}"
    )
    data = response.json()
    
    time = data['hourly'].get('time', [])
    temperature = data['hourly'].get('temperature_2m', [])
    humidity = data['hourly'].get('relative_humidity_2m', [])
    rain = data['hourly'].get('rain', [])

    if not (time and temperature and humidity and rain):
        return {"error": "Incomplete weather data."}

    daily_data = defaultdict(lambda: {'temperature': [], 'humidity': [], 'total_rain': 0, 'rain_times': []})

    for i in range(len(time)):
        date = datetime.fromisoformat(time[i]).date()
        daily_data[date]['temperature'].append(temperature[i])
        daily_data[date]['humidity'].append(humidity[i])
        daily_data[date]['total_rain'] += rain[i]

        if rain[i] > 0:
            daily_data[date]['rain_times'].append({"time": time[i].split('T')[1], "rain": rain[i]})

    result = {}
    for date, values in daily_data.items():
        avg_temp = sum(values['temperature']) / len(values['temperature'])
        avg_humidity = sum(values['humidity']) / len(values['humidity'])
        result[str(date)] = {
            "average_temperature": round(avg_temp, 1),
            "average_humidity": round(avg_humidity, 1),
            "total_rain": round(values['total_rain'], 1),
            "rain_times": values['rain_times']
        }

    return result, data

In [12]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_weather_forecast",
      "description": "Retrieve the weather forecast including average temperature (°C), average relative humidity (%), total rainfall (mm), and rain timing for specified coordinates within a given date range.",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "number",
            "description": "Latitude of the location (in decimal degrees, range: -90 to 90).",
            "minimum": -90,
            "maximum": 90
          },
          "longitude": {
            "type": "number",
            "description": "Longitude of the location (in decimal degrees, range: -180 to 180).",
            "minimum": -180,
            "maximum": 180
          },
          "start_date": {
            "type": "string",
            "description": "Start date for the forecast period (format: YYYY-MM-DD).",
            "pattern": "^\\d{4}-\\d{2}-\\d{2}$"
          },
          "end_date": {
            "type": "string",
            "description": "End date for the forecast period (format: YYYY-MM-DD).",
            "pattern": "^\\d{4}-\\d{2}-\\d{2}$"
          }
        },
        "required": ["latitude", "longitude", "start_date", "end_date"],
        "additionalProperties": False
      },
      "strict": True
    }
  }
]

In [13]:
system_prompt = """
        **Goal:**
        Generate a clear, concise, and human-like weather forecast summary from structured weather data. The summary should use **future tense** and present the information in a natural, easy-to-understand tone.

        **Input Format:**
        The model will receive a JSON object representing the weather forecast. Each key is a date in `YYYY-MM-DD` format, and the corresponding value is an object containing the following attributes:

        - `average_temperature` (float): The average temperature in Celsius.
        - `average_humidity` (float): The average humidity as a percentage.
        - `total_rain` (float): The total rainfall in millimeters.
        - `rain_times` (list): A list of times (in `HH:MM` format) when rain is expected. An empty list means no rain is forecasted.

        **Example Input:**
        ```json
        {
        "2025-03-27": {"average_temperature": 27.6, "average_humidity": 74.7, "total_rain": 0.0, "rain_times": []},
        "2025-03-28": {"average_temperature": 28.9, "average_humidity": 68.4, "total_rain": 0.0, "rain_times": []},
        "2025-03-29": {"average_temperature": 29.2, "average_humidity": 67.4, "total_rain": 0.0, "rain_times": []}
        }
        ```

        **Output Format:**
        A natural language weather forecast summary using future tense.

        **Example Output:**
        "**Weather Forecast (March 27–29, 2025):**
        Expect **warm and dry** weather over these three days. Temperatures will gradually rise from **27.6°C** on March 27 to **29.2°C** by March 29. Humidity will decrease slightly, dropping from **74.7%** to **67.4%**, making the air feel a bit less sticky. **No rain** is expected during this period, so the days should stay **clear and dry** throughout."

        **Instructions for the Model:**
        1. **Identify the Date Range:** Parse the JSON object and determine the range of dates to include in the forecast.

        2. **Summarize Temperature Trends:** Describe how temperatures will change over the forecast period, mentioning specific values and whether they will rise, fall, or remain stable.

        3. **Summarize Humidity Trends:** Comment on how humidity levels will shift, focusing on whether they will increase, decrease, or stay the same.

        4. **Describe Rain Conditions:**
        - If `total_rain` is greater than 0, mention the likelihood of rain and its expected timing using `rain_times`.
        - If no rain is expected, explicitly state that the forecast predicts dry weather.

        5. **Use a Natural Tone:** Use conversational language with phrases like "expect," "will be," and "should remain" to make the summary feel human and approachable.

        6. **Prioritize Clarity and Brevity:** Keep the summary concise while ensuring all key weather trends are covered.

        **Edge Cases:**
        - If the input contains only one date, generate a forecast for that single day.
        - If the temperature or humidity remains unchanged, explicitly state that conditions will be stable.
        - If rain is expected, include specific times when available; otherwise, mention general rainy conditions.

    """

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"What is the weather forecast for {place_of_stay} between {check_in_date} and {check_out_date}?"},
]

In [14]:
completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

In [15]:
completion.model_dump()

{'id': 'chatcmpl-fbdbd193-d8f0-466c-bf68-8d4ff22a6ca3',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_drv1',
      'function': {'arguments': '{"latitude": 12.97, "longitude": 77.59, "start_date": "2025-03-27", "end_date": "2025-03-31"}',
       'name': 'get_weather_forecast'},
      'type': 'function'}]}}],
 'created': 1743040547,
 'model': 'llama-3.3-70b-versatile',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_90c1d253ff',
 'usage': {'completion_tokens': 50,
  'prompt_tokens': 1195,
  'total_tokens': 1245,
  'completion_tokens_details': None,
  'prompt_tokens_details': None,
  'queue_time': 0.101745443,
  'prompt_time': 0.084763192,
  'completion_time': 0.181818182,
  'total_time': 0.266581374},
 'x_groq': {'id': 'req_01jqan02wnekwtqpyv9e

In [16]:
completion.choices[0].message.tool_calls[0].function.arguments

'{"latitude": 12.97, "longitude": 77.59, "start_date": "2025-03-27", "end_date": "2025-03-31"}'

In [17]:
def call_function(name, args):
    if name == "get_weather_forecast":
        return get_weather_forecast(**args)

In [18]:

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result, _ = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [19]:

class WeatherResponse(BaseModel):
    weather_report: str = Field(
        description="A text summary on the weather in natural language based on the weather forcast data")

In [20]:
completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

In [21]:
print(completion_2.choices[0].message.content)

**Weather Forecast (March 27–31, 2025):**
Expect **warm and dry** weather over these five days. Temperatures will be relatively stable, ranging from **26.9°C** to **27.8°C**, with a slight decrease by March 31. Humidity will decrease significantly, dropping from **56.0%** to **38.0%**, making the air feel less sticky, before slightly increasing to **46.3%** by the end of the period. **No rain** is expected during this period, so the days should stay **clear and dry** throughout.


In [22]:
response = requests.get(f"http://localhost:8084/hotels?city=Bengaluru&ratings=%5B5%2C4%2C3%5D")
data = response.json()

In [23]:
def get_reviews_by_city_and_hotel(json_file, city_name, hotel_name):
    # Load JSON data
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Iterate through cities
    for city in data.get("cities", []):
        if city["city"].lower() == city_name.lower():
            # Iterate through hotels
            for hotel in city.get("hotels", []):
                if hotel["hotel_name"].lower() == hotel_name.lower():
                    return hotel.get("reviews", [])

    return []

In [24]:
json_file = '/home/uuj1cob/projects/Ai-Hackathon/accomodation_assistant/knowledge_base/hotel_reviews.json'

# for hotel in data['hotels']:
#     if not hotel['name'].startswith('TEST'):
#         reviews = get_reviews_by_city_and_hotel(json_file, 'Bengaluru', hotel['name'])

#         if reviews:
#             print(f"Hotel name: {hotel['name']}")
#             for i, review in enumerate(reviews):
#                 print(f"Review {i} (posted on {review['date']}):")
#                 # print(f"Reviewer: {review['reviewer_name']}")
#                 # print(f"Rating: {review['rating']}")
#                 print(review['comment'])
#             print("-" * 50)

reviews_string = ""
hotels_list = []
for hotel in data['hotels']:
    if not hotel['name'].startswith('TEST'):
        reviews = get_reviews_by_city_and_hotel(json_file, 'Bengaluru', hotel['name'])

        if reviews:
            hotels_list.append(hotel['name'])
            reviews_string += (f"\nHotel name: {hotel['name']}")
            for i, review in enumerate(reviews):
                reviews_string += (f"\nReview {i} (posted on {review['date']}): {review['comment']}")
            reviews_string += "\n" + "-" * 50

# reviews_dict = {}

# for hotel in data['hotels']:
#     if not hotel['name'].startswith('TEST'):
#         reviews = get_reviews_by_city_and_hotel(json_file, 'Bengaluru', hotel['name'])

#         if reviews:
#             hotels_list.append(hotel['name'])
#             reviews_string += (f"\nHotel name: {hotel['name']}")
#             for i, review in enumerate(reviews):
#                 reviews_string += (f"\nReview {i} (posted on {review['date']}): {review['comment']}")
#             reviews_string += "\n" + "-" * 50

In [25]:
print(reviews_string)


Hotel name: GINGER BANGALORE WHITEFIELD
Review 0 (posted on 2025-02-20): I recently stayed at Ginger Hotels in Whitefield, Bangalore, and had a pleasant experience overall. The location was quite calm and good, making it a comfortable stay away from the city’s hustle. The hotel’s ambience was nice, and the food quality was also good. The rooms were clean and well-maintained, but the size was quite small, fitting just one or two people comfortably. For work-related stays, it was decent, and the pricing was quite reasonable for the services offered. One of the highlights was the excellent room service and the friendly, respectful staff. A special mention to Khushi, who handled everything smoothly and ensured a hassle-free experience. Overall, a good budget-friendly option for short work trips!
Review 1 (posted on 2025-03-01): Nice place to stay with Metro access close by. Staff were kind and courteous. Mr. Samim ensured the room is always clean Buffet option is not available on all days

In [26]:
# Define the Pydantic model for hotel ranking
class HotelRanking(BaseModel):
    hotel_name: str = Field(description=f"Name of the hotel exactly as in {hotels_list}")
    reason: str = Field(description="A short explanation as to why the hotel was chosen for this rank?")
    pros: str = Field(description="A bullet points about the pros of the hotel")
    cons: str = Field(description="A bullet points about the cons of the hotel")

class HotelRankingsResponse(BaseModel):
    rank_1: HotelRanking = Field(description="The name, reason for being assigned first rank, pros and cons of the first ranked hotel")
    rank_2: HotelRanking = Field(description="The name, reason for being assigned second rank, pros and cons of the second ranked hotel")
    rank_3: HotelRanking = Field(description="The name, reason for being assigned third rank, pros and cons of the third ranked hotel")

# System prompt for hotel ranking
system_prompt = f"""
    You are a Hotel Ranking Assistant.
    Your task is to evaluate and rank the hotels based on provided guest reviews.
    After ranking the hotels, choose the top three hotels.
    Safety should be most important criteria when evaluating the hotels.

    Other Evaluation Criteria are as follows,
    1. Service Quality: Friendliness, attentiveness, and responsiveness of staff.
    2. Cleanliness & Maintenance: Cleanliness of rooms, public areas, and general upkeep.
    3. Location & Accessibility: Proximity to key city areas, convenience of transport.
    4. Amenities & Comfort: Quality of rooms, dining options, event spaces, and technology.
    5. Guest Satisfaction: Consistency of positive feedback, handling of issues, and special gestures.

    Ranking Logic:
    - Analyze and compare the hotels across all criteria with safety being the utmost priority.
    - Rank hotels from best to worst, clearly explaining why each hotel holds its position.
    - Highlight both positive aspects (pros) and negative aspects (cons) for each hotel.

    Output should be in below json format:
    {json.dumps(HotelRankingsResponse.model_json_schema(), indent=2)}
"""

In [ ]:
"""    The output should look as follows,
    Rank 1:
    - Hotel Name: The name of the hotel ranked first.
    - Reason for rank: Summarise the reason for its rank
    - Pros: the pros about staying in the hotel
    - Cons: the cons about staying in the hotel

    Rank 2:
    - Hotel Name: The name of the hotel ranked second.
    - Reason for rank: Summarise the reason for its rank
    - Pros: the pros about staying in the hotel
    - Cons: the cons about staying in the hotel

    Rank 3:
    - Hotel Name: The name of the hotel ranked third.
    - Reason for rank: Summarise the reason for its rank
    - Pros: the pros about staying in the hotel
    - Cons: the cons about staying in the hotel"""

In [28]:
completion_reviews = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": f"Based on the following reviews, summarise and choose the top 3 hotels. Reviews:\n{reviews_string}"},
        ],
        response_format={"type": "json_object"}
    )

In [31]:
print(HotelRankingsResponse.model_validate_json(completion_reviews.choices[0].message.content))

rank_1=HotelRanking(hotel_name='THE OBEROI, BANGALORE', reason="Highly praised for its wonderful staff, comfortable rooms, and excellent service, making it a must-stay hotel. Guests appreciated the hotel's beautiful gardens, pool area, and delicious breakfast buffet.", pros="* Beautiful gardens and pool area * Comfortable rooms with an 'old India' feel * Excellent service and attention to detail * Delicious breakfast buffet * Convenient location", cons='* Some guests found the front desk service to be horrible * Room controls via iPad were laggy') rank_2=HotelRanking(hotel_name='JW MARRIOTT HOTEL BENGALURU', reason="Commended for its great location, excellent ambience, and friendly staff. Guests enjoyed the hotel's luxurious atmosphere and convenient access to nearby attractions.", pros='* Great location in the heart of Bengaluru * Excellent ambience and friendly staff * Luxurious atmosphere * Convenient access to nearby attractions', cons='* Some guests experienced horrible service an

In [34]:
print(completion_reviews.choices[0].message.content)

{
   "rank_1": {
      "hotel_name": "THE OBEROI, BANGALORE",
      "reason": "Highly praised for its wonderful staff, comfortable rooms, and excellent service, making it a must-stay hotel. Guests appreciated the hotel's beautiful gardens, pool area, and delicious breakfast buffet.",
      "pros": "* Beautiful gardens and pool area * Comfortable rooms with an 'old India' feel * Excellent service and attention to detail * Delicious breakfast buffet * Convenient location",
      "cons": "* Some guests found the front desk service to be horrible * Room controls via iPad were laggy"
   },
   "rank_2": {
      "hotel_name": "JW MARRIOTT HOTEL BENGALURU",
      "reason": "Commended for its great location, excellent ambience, and friendly staff. Guests enjoyed the hotel's luxurious atmosphere and convenient access to nearby attractions.",
      "pros": "* Great location in the heart of Bengaluru * Excellent ambience and friendly staff * Luxurious atmosphere * Convenient access to nearby attra

In [122]:
completion_reviews_2 = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": reviews_string},
        ],
        response_format=HotelRankingsResponse,
        temperature = 0.7
    )

{ "rank_1": {"hotel_name": "JW Marriott Hotel Bengaluru", "reason": "Excellent service, great location, and luxurious amenities." , "pros": "Great location, excellent service, luxurious amenities, convenient to shopping and restaurants.", "cons": "Room key didn't work initially, breakfast was horrible"}, "rank_2": {"hotel_name": "Marriott Hotel Bengaluru", "reason": "Friendly staff, great hospitality, and good food." , "pros": "Friendly staff, great hospitality, tasty food, convenient location.", "cons": "L2 hall was small for family function, food not 5-star level"}, "rank_3": {"hotel_name": "Zuri Hotel Bengaluru", "reason": "Poor service and subpar amenities." , "pros": "Nice clean hotel, good for family, excellent staff at restaurant.", "cons": "No direct elevator access to basement parking, poor customer service, overcharging."} }
